In [7]:
import pandas as pd

import sys
sys.path.append("/home/paolo/airflow/dags")

from custom_code import custom_lib as cl


fp = cl.path_utils.get_today_remotive()
print(fp)

df = pd.read_json(fp)
df.head()
print(df.category.unique())

sw_df = df[df["category"] == "Software Development"]
sw_df.head()
sw_df.candidate_required_location.unique()
sw_df["location"] = sw_df.candidate_required_location.str.lower()

worldwide_europe_germany = sw_df[
    (sw_df["location"].str.contains("europe"))
    | (sw_df["location"].str.contains("germany"))
    | (sw_df["location"].str.contains("worldwide"))

]

worldwide_europe_germany.head()

currency_api = cl.currency_api.CachedCurrencyAPI()
t = currency_api.convert_usd_to_euros(50.000)
print(t)

/data/remotive/2023-01-23_remotive.json
['Design' 'Software Development' 'All others' 'Human Resources'
 'Finance / Legal' 'DevOps / Sysadmin' 'Product' 'Sales' 'Marketing'
 'Customer Service' 'Data' 'QA' 'Business' 'Writing']
Cache hit, returning JSON from FS
45.9925


/tmp/ipykernel_36498/2708020926.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sw_df["location"] = sw_df.candidate_required_location.str.lower()
